In [1]:
using SpecialFunctions
using Random
using Distributions
using Statistics
using Plots; pgfplotsx()
using LaTeXStrings
using Base.Threads
using StatsBase
using LinearAlgebra
using Trapz
using Roots

# Effective force and resetting rate

In [2]:
# Effective force
function V(y,z)
    return y*exp(-y^2)*exp(-z) / (√z * (exp(-z-y^2) + √(π*z) * erf(√z)) )
end

# Effective resetting rate
function W(y,z)
    return (exp(-z) + √(π*z)*erf(√z)) / (exp(-z-y^2) + √(π*z)*erf(√z))
end

W (generic function with 1 method)

# Resetting Brownian bridge trajectories

In [3]:
# Generate resetting Brownian bridge (RBB) trajectories by integrating the effective Langevin equation.

function run(r,dt,tf)
    D = 1 # Diffusion coefficent
    dx = √(2*dt)
    t = 0:dt:tf
    x = zeros(length(t))
    for i=2:length(t)
        y = x[i-1]/√(4*(tf-t[i]))
        z = r*(tf-t[i])
        if rand() < r*W(y,z)*dt
            x[i] = 0
        else
            x[i] = x[i-1] + dx*randn() - √(4*r)*V(y,z)*dt
        end
    end
    return t, x
end

run (generic function with 1 method)

In [4]:
# Generate one trajectory

r = 10 # resetting rate
dt = 10^(-3) # time increment
tf = 1 # final time
t,x=run(r,dt,tf);

(0.0:0.0010000000000000002:0.9990000000000002, [0.0, -0.014685693929231953, 0.029666429831207833, 0.07015365604654572, 0.026170542351660095, -0.003055083935813405, 0.0006650185633222555, -0.011876022039460967, -0.06421020114049848, -0.1583491086052331  …  -0.09741718307356349, -0.12794608179952377, -0.056906108106663925, -0.15122246194507996, -0.19108655570116967, -0.1392111442991526, -0.12517528121642307, -0.08381145449077904, -0.005090461122941731, -0.06480839661797634])

In [ ]:
# Show trajectory

plot()
for i=1:(length(x)-1)
    if x[i+1]==0
        plot!([t[i],t[i+1]],[x[i],x[i+1]],arrow=true,color=:red,linestyle=:dash,linewidth=3,label="")
    else
        plot!([t[i],t[i+1]],[x[i],x[i+1]],color=:blue,linewidth=2,label="")
    end
end
xlabel!(L"t")
ylabel!(L"x_B(t)")
plot!(grid=false,size=(500,350),tickfontsize=20,guidefontsize=20,legendfontsize=20)
current()